In [1]:
using Revise

In [2]:
@assert Threads.nthreads() > 1

In [3]:
using MeshCatMechanisms
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using LCPSim
using Blink
using LearningMPC
using LearningMPC.Models
using RigidBodySim

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [5]:
atlas_sim = LearningMPC.Models.PlanarAtlas(:simulation);

In [6]:
mvis = MechanismVisualizer(atlas_sim)
open(mvis, Window())
set_configuration!(mvis, configuration(nominal_state(atlas_sim)))

AtlasRobot.packagepath() = "/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/AtlasRobot/deps"


Locally we express box atlas state $x_b$ as a function of full atlas state $x_a$: 

$$
x_b = J_{ba} (x_a - x_{a0}) + x_{b0}
$$

Our goal is to minimize an quadratic cost expressed in terms of the box atlas state: 

$$
\text{minimize } (x_b - x_b^*)^\top Q (x_b - x_b^*) + q^\top x_b + u_b^\top R u_b
$$

Substituting, we get: 

$$
(J_{ba} (x_a - x_{a0}) + x_{b0} - x_b^*)^\top Q (J_{ba} (x_a - x_{a0}) + x_{b0} - x_b^*) + q^\top (J_{ba} (x_a - x_{a0}) + x_{b0}) + u_b^\top R u_b
$$

$$
= x_a^\top (J_{ba}^\top Q J_{ba}) x_a + 2 (-J_{ba} x_{a0} + x_{b0} - x_b^*)^\top Q J_{ba} x_a + q^\top J_{ba} x_a + u_b^\top R u_b + \text{ constant}
$$

Assuming that $q = 0$ for now, what's the analytical solution? 

$$
2 x_a^\top J_{ba}^\top Q J_{ba} + 2(-J_{ba}x_{a0} + x_{b0} - x_b^*)^\top Q J_{ba} = 0
$$

$$
(J_{ba} x_a -J_{ba}x_{a0} + x_{b0} - x_b^*)^\top Q J_{ba} = 0
$$

which is true when $J_{ba}(x_a - x_{a0}) + x_{b0}) = x_b^*$

That constant term comes out to: 

$$
(-J_{ba} x_{a0} + x_{b0} - x_b^*)^\top Q (-J_{ba} x_{a0} + x_{b0} - x_b^*) + q^\top (-J_{ba} x_{a0} + x_{b0})
$$

In [7]:
# atlas_control = LearningMPC.Models.PlanarAtlas(:control);
feet = Dict(:left => findbody(mechanism(atlas_sim), "l_foot_sole"), :right => findbody(mechanism(atlas_sim), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(atlas_sim), "l_hand_mount"), :right => findbody(mechanism(atlas_sim), "r_hand_mount"));
floor = HalfSpace3D(Point3D(root_frame(mechanism(atlas_sim)), 0., 0, 0), FreeVector3D(root_frame(mechanism(atlas_sim)), 0., 0, 1))

xstar = nominal_state(atlas_sim)
Δt = 0.05
nq = num_positions(xstar)
nv = num_velocities(xstar)
qq = fill(0.1, nq)
qq[1] = 10
qq[2] = 100
qq[3] = 500
# qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "r_leg_hpx"))] .= 100
# qq[configuration_range(xstar, findjoint(mechanism(atlas_sim), "l_leg_hpx"))] .= 100
qv = fill(0.01, nv)
qv[velocity_range(xstar, findjoint(xstar.mechanism, "floating_base"))] .= [100, 20, 50]
Q = diagm(vcat(qq, qv))
R = diagm(fill(1e-6, nv))
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, Δt, 
    [Point3D(default_frame(body), 0., 0, 0) for body in 
            findbody.(mechanism(atlas_sim), ["r_foot_sole", "l_foot_sole"])])


(::LQRSolution) (generic function with 2 methods)

In [339]:
mixed_control, lowlevel_controllers = build_mixed_controller(atlas_sim, lqrsol);

contact_state = (true, true)
contact_state = (false, true)
contact_state = (true, false)
contact_state = (false, false)


In [340]:
state = nominal_state(atlas_sim)
# set_velocity!(state, findjoint(mechanism(atlas_sim), "floating_base"), [-0., 0, 0])
mixed_control(zeros(num_velocities(state)), 0.0, state)

27-element Array{Float64,1}:
   1.31115e-6 
   5.87668e-6 
  -1.615e-6   
  -1.31903    
   0.49937    
  40.4864     
  40.1485     
  -0.000207508
   0.000413095
 -96.9099     
 -95.6951     
  15.5587     
 -15.3948     
   ⋮          
  -0.0017269  
  -0.00247717 
   1.28171    
  -1.34042    
   4.09769    
  -4.05687    
  -0.00172817 
  -0.00247972 
   0.314055   
  -0.324302   
   0.00039535 
  -0.000387549

In [341]:
state = nominal_state(atlas_sim)
set_velocity!(state, findjoint(mechanism(atlas_sim), "floating_base"), [-0.2, 0, 0])
final_time = 5.0

composed = PeriodicController(similar(velocity(state)), 0.01, compose(mixed_control, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer, damper)

dynamics = Dynamics(mechanism(atlas_sim), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback)) # CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
# @time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
RigidBodySim.animate(mvis, mvis.state, sol)

180.674837 seconds (1.27 M allocations: 86.076 MiB, 0.03% gc time)


In [342]:
RigidBodySim.animate(mvis, mvis.state, sol)